In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

print(tf.__version__)

2.4.1


In [3]:
import pandas as pd
data = pd.read_csv("../input/twitter-impact1/Twitter.csv")

In [4]:
data.isna().sum()

Unnamed: 0              0
Id                      0
Post Contet             0
Sentiment score         0
Post Length             0
Hashtag count           0
Content URL count       0
Tweet count             0
Followers count         0
Listed Count            0
Media Type              0
Published DateTime      0
Mentions Count          0
Post author verified    0
Likes                   0
Shares                  0
Comments                0
Impact                  0
dtype: int64

In [5]:
data.drop(['Unnamed: 0', "Id","Post Contet","Published DateTime"], inplace=True, axis=1)

In [6]:
X = data.iloc[:, :13]
X.head()

,Sentiment score,Post Length,Hashtag count,Content URL count,Tweet count,Followers count,Listed Count,Media Type,Mentions Count,Post author verified,Likes,Shares,Comments
0,0.0,17.0,0.0,0.0,1660.0,133.0,0,TEXT,0.0,0.0,0.0,0.0,0.0
1,19.3,84.0,0.0,1.0,318924.0,1428273.0,5454,TEXT,0.0,1.0,10.0,1.0,0.0
2,0.0,78.0,0.0,0.0,39976.0,1560.0,0,TEXT,0.0,0.0,0.0,0.0,0.0
3,0.0,284.0,2.0,0.0,8045.0,27191.0,26,TEXT,0.0,1.0,460.0,219.0,4.0
4,-20.0,80.0,0.0,1.0,210996.0,2385.0,0,IMAGE,0.0,0.0,0.0,0.0,0.0


In [7]:
y = data.iloc[:, 13:]
y.head()

,Impact
0,0.0
1,110.0
2,0.0
3,6830.0
4,0.0


In [8]:
X = pd.get_dummies(X, columns = ['Media Type'])
X.head()

,Sentiment score,Post Length,Hashtag count,Content URL count,Tweet count,Followers count,Listed Count,Mentions Count,Post author verified,Likes,Shares,Comments,Media Type_IMAGE,Media Type_TEXT,Media Type_VIDEO
0,0.0,17.0,0.0,0.0,1660.0,133.0,0,0.0,0.0,0.0,0.0,0.0,0,1,0
1,19.3,84.0,0.0,1.0,318924.0,1428273.0,5454,0.0,1.0,10.0,1.0,0.0,0,1,0
2,0.0,78.0,0.0,0.0,39976.0,1560.0,0,0.0,0.0,0.0,0.0,0.0,0,1,0
3,0.0,284.0,2.0,0.0,8045.0,27191.0,26,0.0,1.0,460.0,219.0,4.0,0,1,0
4,-20.0,80.0,0.0,1.0,210996.0,2385.0,0,0.0,0.0,0.0,0.0,0.0,1,0,0


In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [11]:
model = keras.Sequential([
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1, activation = "relu")
  ])

model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001),
                metrics = "mean_absolute_error")


In [12]:
input_shape = X.shape  
model.build(input_shape) 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (50000, 64)               1024      
_________________________________________________________________
dense_1 (Dense)              (50000, 64)               4160      
_________________________________________________________________
dense_2 (Dense)              (50000, 1)                65        
Total params: 5,249
Trainable params: 5,249
Non-trainable params: 0
_________________________________________________________________


In [13]:
%%time
history = model.fit(
    X_train, y_train, 
    epochs=20,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

Epoch 1/20
1000/1000 [==============================] - 3s 2ms/step - loss: 0.5246 - mean_absolute_error: 0.5246 - val_loss: 0.5431 - val_mean_absolute_error: 0.5431
Epoch 2/20
1000/1000 [==============================] - 2s 2ms/step - loss: 0.5224 - mean_absolute_error: 0.5224 - val_loss: 0.5431 - val_mean_absolute_error: 0.5431
Epoch 3/20
1000/1000 [==============================] - 2s 2ms/step - loss: 0.5217 - mean_absolute_error: 0.5217 - val_loss: 0.5431 - val_mean_absolute_error: 0.5431
Epoch 4/20
1000/1000 [==============================] - 2s 2ms/step - loss: 0.5206 - mean_absolute_error: 0.5206 - val_loss: 0.5431 - val_mean_absolute_error: 0.5431
Epoch 5/20
1000/1000 [==============================] - 2s 2ms/step - loss: 0.5269 - mean_absolute_error: 0.5269 - val_loss: 0.5431 - val_mean_absolute_error: 0.5431
Epoch 6/20
1000/1000 [==============================] - 2s 2ms/step - loss: 0.5244 - mean_absolute_error: 0.5244 - val_loss: 0.5431 - val_mean_absolute_error: 0.5431
Epoc

In [14]:
test_results = {}
test_results['linear_model'] = model.evaluate(
    X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.5268 - mean_absolute_error: 0.5268
